### Pyspark Handling Missing Values
- Dropping Columns
- Dropping Rows
- Various Parameter in Dropping functionalities
- Handling Missing values by Mean, Median and Mode

In [24]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config('spark.port.host', 'localhost').appName('practice').getOrCreate()

In [25]:
df = spark.read.csv('table3.csv', header=True, inferSchema=True)

In [26]:
df.show()

+-----+----+----------+------+
| Name| Age|Experience|Salary|
+-----+----+----------+------+
|Alice|  30|         5| 60000|
|  Bob|  28|         3| 50000|
|Carol|  35|         8| 75000|
|David|  32|      NULL| 65000|
|Emily|  27|         2| 48000|
|Frank|  40|        12| 90000|
|Grace|NULL|         4|  NULL|
|Henry|  33|         7| 70000|
| NULL|  31|         5| 62000|
| Jack|  26|         1|  NULL|
+-----+----+----------+------+



In [27]:
# Drop the columns
drop_df = df.drop('Name')
drop_df.show()

+----+----------+------+
| Age|Experience|Salary|
+----+----------+------+
|  30|         5| 60000|
|  28|         3| 50000|
|  35|         8| 75000|
|  32|      NULL| 65000|
|  27|         2| 48000|
|  40|        12| 90000|
|NULL|         4|  NULL|
|  33|         7| 70000|
|  31|         5| 62000|
|  26|         1|  NULL|
+----+----------+------+



In [28]:
# Drop the NULL values
drop_null_df = df.na.drop()
drop_null_df.show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
|Alice| 30|         5| 60000|
|  Bob| 28|         3| 50000|
|Carol| 35|         8| 75000|
|Emily| 27|         2| 48000|
|Frank| 40|        12| 90000|
|Henry| 33|         7| 70000|
+-----+---+----------+------+



In [29]:
# How (all) contraint
all_drop_null_df = df.na.drop(how='all')
all_drop_null_df.show()

+-----+----+----------+------+
| Name| Age|Experience|Salary|
+-----+----+----------+------+
|Alice|  30|         5| 60000|
|  Bob|  28|         3| 50000|
|Carol|  35|         8| 75000|
|David|  32|      NULL| 65000|
|Emily|  27|         2| 48000|
|Frank|  40|        12| 90000|
|Grace|NULL|         4|  NULL|
|Henry|  33|         7| 70000|
| NULL|  31|         5| 62000|
| Jack|  26|         1|  NULL|
+-----+----+----------+------+



In [30]:
# DROP - How (any) contraint
any_drop_null_df = df.na.drop(how='any')
any_drop_null_df.show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
|Alice| 30|         5| 60000|
|  Bob| 28|         3| 50000|
|Carol| 35|         8| 75000|
|Emily| 27|         2| 48000|
|Frank| 40|        12| 90000|
|Henry| 33|         7| 70000|
+-----+---+----------+------+



In [31]:
# DROP - threshold (it will remove if the number of non null values given in threshold is low).
drop_threshold_df = df.na.drop(how='any', thresh=3)
drop_threshold_df.show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
|Alice| 30|         5| 60000|
|  Bob| 28|         3| 50000|
|Carol| 35|         8| 75000|
|David| 32|      NULL| 65000|
|Emily| 27|         2| 48000|
|Frank| 40|        12| 90000|
|Henry| 33|         7| 70000|
| NULL| 31|         5| 62000|
| Jack| 26|         1|  NULL|
+-----+---+----------+------+



In [32]:
# DROP - Subset (want to filter out NULL values of particular column then we will use subset).
drop_subset_df = df.na.drop(how='any', subset=['Experience'])
drop_subset_df.show()

+-----+----+----------+------+
| Name| Age|Experience|Salary|
+-----+----+----------+------+
|Alice|  30|         5| 60000|
|  Bob|  28|         3| 50000|
|Carol|  35|         8| 75000|
|Emily|  27|         2| 48000|
|Frank|  40|        12| 90000|
|Grace|NULL|         4|  NULL|
|Henry|  33|         7| 70000|
| NULL|  31|         5| 62000|
| Jack|  26|         1|  NULL|
+-----+----+----------+------+



In [33]:
# Filling the missing value in respect of its datatype.
# Subset is the list of columns where we want to fill the missing values
# fill_df = df.na.fill(-1)
fill_df = df.na.fill(-1,  subset = ['Experience', 'Salary'])
fill_df = fill_df.na.fill('Missing Values')
fill_df.show()

+--------------+----+----------+------+
|          Name| Age|Experience|Salary|
+--------------+----+----------+------+
|         Alice|  30|         5| 60000|
|           Bob|  28|         3| 50000|
|         Carol|  35|         8| 75000|
|         David|  32|        -1| 65000|
|         Emily|  27|         2| 48000|
|         Frank|  40|        12| 90000|
|         Grace|NULL|         4|    -1|
|         Henry|  33|         7| 70000|
|Missing Values|  31|         5| 62000|
|          Jack|  26|         1|    -1|
+--------------+----+----------+------+



In [38]:
# Filling the NULL values in `Experience` column with mean of `Experience` values
from pyspark.ml.feature import Imputer
imputer = Imputer(
    inputCols=['Age', 'Experience', 'Salary'],
    outputCols=["{}_imputed".format(m) for m in ['Age', 'Experience', 'Salary']]
).setStrategy('median')

fill_imputer_df = imputer.fit(df).transform(df)
fill_imputer_df.show()

+-----+----+----------+------+-----------+------------------+--------------+
| Name| Age|Experience|Salary|Age_imputed|Experience_imputed|Salary_imputed|
+-----+----+----------+------+-----------+------------------+--------------+
|Alice|  30|         5| 60000|         30|                 5|         60000|
|  Bob|  28|         3| 50000|         28|                 3|         50000|
|Carol|  35|         8| 75000|         35|                 8|         75000|
|David|  32|      NULL| 65000|         32|                 5|         65000|
|Emily|  27|         2| 48000|         27|                 2|         48000|
|Frank|  40|        12| 90000|         40|                12|         90000|
|Grace|NULL|         4|  NULL|         31|                 4|         62000|
|Henry|  33|         7| 70000|         33|                 7|         70000|
| NULL|  31|         5| 62000|         31|                 5|         62000|
| Jack|  26|         1|  NULL|         26|                 1|         62000|